In [2]:
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor

In [79]:
df1 = pd.read_csv('datasets/db_waw_flats.csv', parse_dates = ['today', 'publication_date'])
df2 = pd.read_csv('datasets/db_waw_flats_17_06.csv', parse_dates = ['today', 'publication_date'])
df3 = pd.read_csv('datasets/db_waw_flats_27_6.csv', parse_dates = ['today', 'publication_date'])

In [194]:
common_df = pd.concat([df1,df2,df3], ignore_index=True)
common_df

,price,area,rooms,renovation,floor,market,elevator,street,district,balcony,terrace,garden,parking,central_heating,seller,blok,cena_m,today,publication_date
0,1152000.0,72.00,3.0,3,2,2,1,ul. gen. tadeusza pełczyńskiego,bemowo,1,0,0,1,0,2,1,16000.000000,2023-05-25,2023-05-14 19:36:44
1,1790000.0,115.41,4.0,3,2,2,1,ul. obrzeżna,mokotów,0,1,0,1,1,1,0,15509.921151,2023-05-25,2023-05-19 15:27:31
2,610000.0,54.20,3.0,3,2,2,1,ul. magiczna,białołęka,1,0,0,1,0,2,1,11254.612546,2023-05-25,2023-05-06 19:36:14
3,799000.0,61.10,3.0,3,2,2,1,ul. marywilska,białołęka,1,0,0,1,0,2,1,13076.923077,2023-05-25,2023-05-22 19:37:02
4,1040000.0,46.00,2.0,3,2,2,1,ul. żelazna,wola,1,0,0,0,1,2,0,22608.695652,2023-05-25,2023-05-06 19:33:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27857,1479400.0,113.80,4.0,3,1,2,1,ul. meander,ursynów,1,0,0,1,1,1,1,13000.000000,2023-06-27,2015-10-27 20:02:18
27858,7520000.0,320.00,4.0,3,2,2,1,unknown,wola,1,1,0,1,1,1,0,23500.000000,2023-06-27,2015-06-23 20:17:44
27859,2700000.0,200.00,8.0,1,1,2,0,ul. łotewska,praga-południe,0,0,0,0,0,1,0,13500.000000,2023-06-27,2014-11-10 12:05:12
27860,565000.0,40.17,2.0,2,2,1,1,ul. herbu oksza,ursus,1,0,0,1,1,2,1,14065.222803,2023-06-27,2023-06-26 13:20:05


In [195]:
common_df.drop(columns=['today']).drop_duplicates(keep='last', inplace=True)

In [196]:
cols_x = ['district', 'area', 'rooms', 'renovation', 'floor', 'balcony', 'terrace',
          'garden', 'parking', 'central_heating', 'market', 'seller', 'blok', 'elevator']
col_y = ['price']

In [197]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
le = preprocessing.LabelEncoder()
le.fit(common_df['district'])
common_df['district'] = le.transform(common_df['district'])

X_train, X_test, y_train, y_test = train_test_split(common_df.dropna()[cols_x], common_df.dropna()['price'], test_size=0.25, random_state=10)

In [199]:
from sklearn.tree import DecisionTreeRegressor
est = DecisionTreeRegressor(random_state=0, max_depth=20)
model = AdaBoostRegressor(estimator=est, random_state=0, n_estimators=50, learning_rate=0.05, loss='square')
model.fit(X_train, y_train)
preds = model.predict(X_test)

In [200]:
print('percentage error', mean_absolute_percentage_error(y_test, preds))
print('absolute error', mean_absolute_error(y_test, preds))
print('root squared error', mean_squared_error(y_test, preds, squared=False))

percentage error 0.045625887763010554
absolute error 55998.75087417239
root squared error 295011.6512383968


In [181]:
import pickle
filename = 'adaboost_regressor.pkl'
pickle.dump(model, open(filename, 'wb'))

In [257]:
load_model = pickle.load(open(filename, 'rb'))
load_model.predict([[1, 65, 3, 3, 1, 1, 0, 0, 1, 1, 2, 2, 1, 1]])

/home/arsen/.local/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but AdaBoostRegressor was fitted with feature names
  warnings.warn(


array([773900.])

In [212]:
le.transform(['białołęka'])

array([1])

In [205]:
pickle.dump(le, open('district_encode', 'wb'))